In [1]:
import argparse
import pickle
from pathlib import Path

import fraud_eagle as feagle
import fraudar
import networkx as nx
import numpy as np
import pandas as pd
import rsd

from rev2 import rev2compute
from utils import load_data, split_data_by_time

import seaborn as sns
import matplotlib.pyplot as plt

import sklearn.metrics

In [2]:
def get_results(data_name="alpha", alg="fraudar", attack_name="ilp_attack", budget=100, frac=0.2):
    data_nw_df, data_gt_df = load_data(data_name=data_name)
    p = Path(f"../res/{attack_name}/{alg}-{data_name}/{budget}-{frac}.pkl")
    if not p.exists():
        print(p)
        return None
    with open(p, "rb") as fp:
        scores = pickle.load(fp)
        scores = dict(zip(range(len(scores)), scores))
        # 1 means fraudster, 0 means benign
        scores["truth"] = dict(zip(data_gt_df["id"], (1-data_gt_df["label"])/2))
    # df = pd.DataFrame({sp: {u: scores[sp][u] for u in scores[sp] if u in data_gt_df["id"].tolist()} for sp in scores})
    df = pd.DataFrame({sp: {u: scores[sp][u] for u in scores[sp]} for sp in scores})
    df.loc[[u[:5]=="usock" for u in df.index.tolist()], "truth"] = 1
    df.loc[[u[:6]=="udummy" for u in df.index.tolist()], "truth"] = 1
    return df

In [3]:
def get_metrics(df_res, splits=4, K=None, th=0.5):
    if df_res is None:
        return None
    ret = {}
    for sp in range(splits):
        # df_res["truth"] = df_res["truth"].fillna(1)
        df_res.loc[[u[:5]=="usock" for u in df_res.index.tolist()], "truth"] = 1
        df_res.loc[[u[:6]=="udummy" for u in df_res.index.tolist()], "truth"] = 1
        df = df_res[[sp, "truth"]].dropna()
        df[sp]
        t = df["truth"].sum() if K is None else K
        t_frac = t / df.shape[0]
        df["pred"] = df[sp] >= th

        ret[sp] = {
            "precision": sklearn.metrics.precision_score(y_true=df["truth"], y_pred=df["pred"]),
            "recall": sklearn.metrics.recall_score(y_true=df["truth"], y_pred=df["pred"]),
            "f1": sklearn.metrics.f1_score(y_true=df["truth"], y_pred=df["pred"])
        }
    return ret

In [4]:
alg_list = ["fraudar", "rsd", "rev2"][:3]
data_list = ["alpha", "otc", "amazon", "epinions"][3:]
budget_list = [100.0, 200.0, 300.0, 400.0]
frac_list = [0.0, 0.2, 0.4, 0.6, 0.8]

In [5]:
attack_res = {
    (d, a, b, f): get_results(data_name=d, alg=a, attack_name="ilp_attack", budget=b, frac=f)
    for d in data_list for a in alg_list
    for b in budget_list for f in frac_list
}

../res/ilp_attack/rsd-epinions/200.0-0.6.pkl
../res/ilp_attack/rsd-epinions/300.0-0.0.pkl
../res/ilp_attack/rsd-epinions/300.0-0.2.pkl
../res/ilp_attack/rsd-epinions/300.0-0.4.pkl
../res/ilp_attack/rsd-epinions/300.0-0.6.pkl
../res/ilp_attack/rsd-epinions/400.0-0.0.pkl


In [ ]:
# attack_res[("epinions", "fraudar", 400.0, 0.2)].to_csv("ar.csv")

In [6]:
th_dict = {
    "fraudar": 0.5,
    "rsd": 1.2,
    "rev2": 0.1,
}
dict_met = {
    (d, a, b, f): pd.DataFrame.from_dict(get_metrics(df_res=attack_res[(d, a, b, f)], splits=4, K=None, th=th_dict[a]))
    for d, a, b, f in attack_res
}
df_met = pd.concat(dict_met, keys=dict_met.keys())
display(df_met)

0         1         2         3
epinions fraudar 100.0 0.0 precision  0.016245  0.018277  0.021121  0.022312
                           recall     0.168558  0.168385  0.202952  0.235043
                           f1         0.029634  0.032974  0.038261  0.040756
                       0.2 precision  0.016245  0.018304  0.021121  0.022312
                           recall     0.165342  0.161716  0.194346  0.223577
...                                        ...       ...       ...       ...
         rev2    400.0 0.6 recall     0.124836  0.073563  0.065060  0.076720
                           f1         0.155229  0.115942  0.100935  0.112840
                       0.8 precision  0.205184  0.273504  0.225000  0.213235
                           recall     0.117429  0.066253  0.058315  0.068075
                           f1         0.149371  0.106667  0.092624  0.103203

[162 rows x 4 columns]

In [7]:
def get_alg_data_met(df_met, alg="rev2", data="alpha", met="f1"):
    df_met.index = df_met.index.set_names(["data", "algo", "budget", "frac", "metrics"])
    rdf = df_met.mean(axis=1).to_frame().reset_index().pivot(index=["data", "algo", "frac", "metrics"], columns=["budget"]).loc[(data, alg, slice(None), met)].droplevel(0, axis=1)#.droplevel([0, 1, 3], axis=0)
    rdf.index = [f"baseline-{i}" if i > 0 else "no-attack" for i in rdf.index]
    return rdf

In [13]:
get_alg_data_met(df_met, alg="rsd", data="epinions", met="f1")

budget,100.0,200.0,300.0,400.0
no-attack,0.320685,0.322658,NaN,NaN
baseline-0.2,0.317282,0.315419,NaN,0.309493
baseline-0.4,0.312042,0.306787,NaN,0.294007
baseline-0.6,0.306785,NaN,NaN,0.279000
baseline-0.8,0.305899,0.292035,0.28042,0.268877
